In [36]:
import ccxt, pandas as pd, matplotlib.pyplot as plt, sys, indicateur_techniques as ind
sys.path.append('../Ohlcvplus')
from ohlcv import OhlcvPlus

In [134]:
class Backtest:
    def __init__(self, capital):
        self.capital = capital
        self.position = None

        self.all_position = pd.DataFrame({
			"open_pos" : [],
            "close_pos" : [],
			"mode" : [],
			"open" : [],
			"close" : []
		})

    def load_data(self, symbol='BTC/USDT', time='30m', length=500, sinces='2023-01-01 00:00:00'):
		#telecharcgement des donnée ohlcv
        ohlcvp = OhlcvPlus(ccxt.binance(), database_path='data.db')
        self.data = ohlcvp.load(market=symbol, timeframe=time, since=sinces, limit=length, update=True, verbose=True, workers=100)
        self.close = self.data.close
    
    def copie_data(self, df):
        self.data = df
        self.close = self.data.close
    
    def open_pos(self, taille:int, pos:int):
        if self.position is None:
            self.position = [self.close[pos], taille / self.close[pos], pos]
            print("open", pos)
    
    def close_pos(self, pos:int):
        if self.position is not None:
            self.capital += (self.close[pos] - self.position[0]) * self.position[2]
            print('close', pos)
            print(self.capital)

            add_ligne = pd.DataFrame({
                    "open_pos" : [self.position[2]],
                    "close_pos" : [pos],
					"mode" : ["long"],
					"open" : [self.position[0]],
					"close" : [self.close[pos]]
				})
            self.all_position = pd.concat([self.all_position, add_ligne])
            self.position = None
    
    def graphique(self, *args:list):
        plt.close()
        if len(args) != 1:
            fig, axs = plt.subplots(len(args))
            axs[0].plot(self.close)
            for i in range(len(args)):
                for ind in args[i]:
                    axs[i].plot(ind)
            
            for row in range(len(self.all_position)):
                x = self.all_position[open_pos][row]
                y = self.close[x]
                axs[0].plot(x, y, marker='^', markersize=10, color='g')
                
                x = self.all_position[close_pos][row]
                y = self.close[x]
                axs[0].plot(x, y, marker='v', markersize=10, color='r')

        else: 
            fig, axs = plt.subplots()
            axs.plot(self.close)
            for ind in args[0]:
                axs.plot(ind)


    def trier_signal(self, series):
        result = []
        current_value = False
        for value in series:
            if value == current_value:
                result.append(False)
            else:
                current_value = value
                result.append(value)
        result[0] = False
        return result

In [140]:
%matplotlib
test = Backtest(1000)
test.copie_data(pd.read_csv('data_btc.csv'))
test.open_pos()

Using matplotlib backend: TkAgg
